Final Presentation Notebook

Datasets:

https://data.wprdc.org/dataset/neighborhoods-with-snap-data/resource/bce22c26-9d3e-4e3f-8405-a35c4b7765b6

https://data.wprdc.org/dataset/arrest-data/resource/e03a89dd-134a-4ee8-a2bd-62c40aeebc6f

https://data.wprdc.org/dataset/pbp-fire-arm-seizures/resource/e967381d-d7e9-48e3-a2a2-39262f7fa5c4

Our process of finding the best neighborhood involved finding the ones that are the safest. We used datasets that had information including arrest information, crime and police reports, and gun seizures. Each neighborhood has a count of the things we are measuring, and we combined them all into one metric using all 3 datasets.

In [ ]:
import pandas as pd
from math import isnan
snap = pd.read_csv('snapdata.csv')
seizures = pd.read_csv('firearm_seizures.csv')
arrest = pd.read_csv('arrestdata.csv')

#THIS IS FOR THE FIRST DATASET "SNAP DATA"
neighborhoodCrimes = {}
neighborhoodReports = {}

#Calculating the number of serious crimes committed in each neighborhood using the SNAP data

for index,row in snap.iterrows():
    name = row["Neighborhood_2010_HOOD"]
    murders = row["SNAP_All_csv__Murder__2010_"]
    rape = row["SNAP_All_csv__Rape__2010_"]
    robbery = row["SNAP_All_csv__Robbery__2010_"]
    aggAsslt = row["F_Agr__Assault__2010_"]
    burglary = row["SNAP_All_csv__Burglary__2010_"]
    autoTheft = row["SNAP_All_csv__Auto_Theft__2010_"]

#Crimes being calculated /\
    
    crimes = murders + rape + robbery + aggAsslt + burglary + autoTheft
    
    if name in neighborhoodCrimes:
        neighborhoodCrimes[name] += crimes
    else:
        neighborhoodCrimes[name] = crimes

#Crimes being reported in each neighborhood
    
    policeReports = row["SNAP_All_csv__Part_1__Major_Cri"]
    policeReports2 = row["SNAP_All_csv__Part_2_Reports__2"]
    policeReports3 = row["SNAP_All_csv__Other_Police_Repo"]
    
    totalReports = policeReports + policeReports2 + policeReports3

    if name in neighborhoodReports:
        neighborhoodReports[name] += totalReports
    else:
        neighborhoodReports[name] = totalReports

#Combining the two datasets into one total incidents dictionary
totalIncidents = {}
for neighborhood in neighborhoodReports:
    if neighborhood in totalIncidents:
        totalIncidents[neighborhood] += neighborhoodReports[neighborhood] + neighborhoodCrimes[neighborhood]
    else:
        totalIncidents[neighborhood] = neighborhoodReports[neighborhood] + neighborhoodCrimes[neighborhood]

#Getting maximum number of crimes to use as a "worst case" so that we can compare neighborhoods
worst = totalIncidents[max(totalIncidents, key=totalIncidents.get)]

#Using our total incidents dictionary and worst neighborhood variable to put each neighborhood on a scale, with 0 being the least incidents and 100 being the most
bestNeighborhood = {}
for neighborhood in totalIncidents:
    if neighborhood not in bestNeighborhood:
        val = round((totalIncidents[neighborhood]) / worst * 100, 2)
        bestNeighborhood[neighborhood] = val

#Sorting from least crime to most
bestNeighborhood = dict(sorted(bestNeighborhood.items(), key=lambda item: item[1]))

source1 = {}
for i in bestNeighborhood:
    if i not in scaled:
        val = round(100 - bestNeighborhood[i], 2)
        source1[i] = val 

#THIS IS FOR THE SECOND DATASET "FIREARM SEIZURES"
neighborhoods = {}

#counting number of seizures per neighborhoods
for index,row in seizures.iterrows():
    name = row["neighborhood"]
    total = row["total_count"]

    if name in neighborhoods:
        neighborhoods[name] += total
    else:
        neighborhoods[name] = total

#removing the values that didn't have a neighborhood specified in the csv
hoods = filter(lambda k: not pd.isna(k), neighborhoods)
hoods = {k: neighborhoods[k] for k in neighborhoods if not pd.isna(k)}

#finding the most seizures in a neighborhoods
mostSeizures = hoods[max(hoods, key=hoods.get)]

#sorting from least seizures to most
sortedHoods = dict(sorted(hoods.items(), key=lambda item: item[1]))

weights = {}
for hood in sortedHoods:
    if hood not in weights:
        val = round((sortedHoods[hood]) / mostSeizures * 100, 2)
        weights[hood] = val

source2 = {}
for i in weights:
    if i not in organized:
        val = 100 - weights[i]
        source2[i] = val


#THIS IS FOR THE THIRD DATASET "ARREST DATA"
